This notebook is used to determine which strategy is the best for any particular asset.

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/starting_portfolio_2months.csv")

Below, we use a longer time period just for the calculation of technical indicators s.t. we have values from our desired day 1 of backtesting. We will only run backtesting on a shorter time frame.

In [3]:
import yfinance as yf

start_date = "2023-03-01"  # Longer time period for calculation of technical indicators
true_start_date = "2025-01-01"  # True first day of backtesting
end_date = "2025-03-01"  # Last day is exclusive

portfolio = {}
for asset, weight in list(map(list, df.values)):
    portfolio[asset] = {
        "data": yf.Ticker(asset).history(start=start_date, end=end_date, actions=False),
        "weight": weight,
    }

In [4]:
# Commission fee based on Webull, which is known for low commission fees:
# https://www.webull.com.sg/pricing
# Regular and Extended Hours (04:00 - 20:00 EST)
# 0.025%*Total Trade Amount (Min. USD 0.50)

def commission(order_size, price):
    return max(0.5, abs(order_size) * price * 0.00025)

We calculate all the technical indicators required for all the strategies first before truncating the data. This is so that we have data on e.g. 50-day moving averages even on our desired day 1, but we only run backtesting from day 1 onwards.

In [5]:
import pandas_ta as ta


def calculate_technical_indicators(df):
    """
    We calculate all the technical indicators required for all the strategies first
    before truncating the data. This is so that we have data on e.g. 50-day moving
    averages even on our desired day 1, but we only run backtesting from day 1 onwards.
    """

    # Core indicators
    df["EMA_50"] = ta.ema(df["Close"], length=50)
    df["EMA_200"] = ta.ema(df["Close"], length=200)
    df["RSI"] = ta.rsi(df["Close"], length=14)
    df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=7)

    # Bollinger Bands of length 20
    bbands = ta.bbands(df["Close"], length=20)
    bbands = bbands.rename(
        columns={
            "BBU_20_2.0": "Upper_Band",
            "BBM_20_2.0": "Middle_Band",
            "BBL_20_2.0": "Lower_Band",
            "BBB_20_2.0": "Band_Width",
            "BBP_20_2.0": "Percent_B",
        }
    )

    # Bollinger Bands of length 200
    bbands200 = ta.bbands(df["Close"], length=200)
    bbands200 = bbands200.rename(
        columns={
            "BBU_200_2.0": "Upper_Band_200",
            "BBM_200_2.0": "Middle_Band_200",
            "BBL_200_2.0": "Lower_Band_200",
            "BBB_200_2.0": "Band_Width_200",
            "BBP_200_2.0": "Percent_B_200",
        }
    )

    # MACD
    macd = ta.macd(df["Close"])
    macd = macd.rename(
        columns={
            "MACD_12_26_9": "MACD",
            "MACDh_12_26_9": "Histogram",
            "MACDs_12_26_9": "Signal",
        }
    )

    # Miscellaneous
    df["SMA_20"] = ta.sma(df["Close"], length=20)
    df["STD_20"] = ta.stdev(df["Close"], length=20)
    df["SMA_Volume_10"] = ta.sma(df["Volume"], length=10)
    df["Momentum"] = ta.mom(df["Close"], length=10)

    df = df.join([bbands, bbands200, macd])

    return df

In [6]:
from backtesting import Backtest


def run_backtest(asset, strategy, plot=False):
    data = portfolio[asset]["data"]
    data = calculate_technical_indicators(portfolio[asset]["data"])
    data.index = data.index.values.astype("datetime64[D]")
    data = data.loc[data.index >= true_start_date]
    cash = 1000000 * portfolio[asset]["weight"]
    if asset.endswith("-USD"):  # Crypto
        # https://www.webullpay.com/
        # No Direct Fees: Webull Pay does not charge separate fees for trading cryptocurrencies.
        # Instead, a 1% spread (100 basis points) is included in the buying and selling prices
        # of the crypto assets. This means the cost is integrated into the price you pay or
        # receive when trading.
        bt = Backtest(data, strategy, cash=cash, spread=0.01, finalize_trades=True)
    else:
        bt = Backtest(
            data, strategy, cash=cash, commission=commission, finalize_trades=True
        )
    stats = bt.run()
    if plot:
        bt.plot()
    return bt, stats

## All assets and all strategies

In [7]:
from strategies.backtest.larry_williams_price_action import LarryWilliamsPriceAction
from strategies.backtest.macd_bollinger_bands_mean_reversion import (
    MACDBollingerBandsMeanReversion,
)
from strategies.backtest.mean_reversion import MeanReversion
from strategies.backtest.michael_harris_price_action import MichaelHarrisPriceAction
from strategies.backtest.momentum import Momentum
from strategies.backtest.rsi_divergence import RSIDivergence
from strategies.backtest.scalping import Scalping
from strategies.backtest.volume_spike_reversal import VolumeSpikeReversal
from strategies.backtest.bollinger_bands_breakout import BollingerBandsBreakout

strategies = [
    BollingerBandsBreakout,
    LarryWilliamsPriceAction,
    MACDBollingerBandsMeanReversion,
    MeanReversion,
    MichaelHarrisPriceAction,
    Momentum,
    RSIDivergence,
    Scalping,
    VolumeSpikeReversal,
]

cols = []

for asset in portfolio:
    for strategy in strategies:
        bt, stats = run_backtest(asset, strategy, plot=False)
        col = stats.drop(["_equity_curve", "_trades"])
        col["Asset"] = asset
        col["Weight"] = portfolio[asset]["weight"]
        cols.append(col)

# NOTE: Many instances of broker cancelling the relative-sized order due to insufficient margin.

In [8]:
results = pd.concat(cols, axis=1).transpose()
results = results.rename(columns={"_strategy":"Strategy"})

In [ ]:
results.to_csv("../data/experiments/asset_strategies_2_months_with_tpsl_new_exit_conditions.csv", index=False)

## Example filtering on results

In [ ]:
results.columns

In [10]:
idx = results.groupby("Asset")["Return [%]"].idxmax()
results.loc[idx]